# 1. Introduction & Motivation
Let's start by understanding where we are and what we aim to achieve. In this course, we'll explore the concept of tool wear in manufacturing. Tool wear happens when cutting tools gradually deteriorate due to regular use. This deterioration affects both product quality and productivity. It leads to inaccuracies in dimensions, rough surfaces, defects in products, and increased cutting forces. Managing tool wear is crucial for improving efficiency and maintaining quality in manufacturing processes.

## 1.1. Tool Wear
Flank wear develops as a result of the friction between the tool's flank face and the surface of the workpiece being machined. This friction causes the cutting edge to deteriorate, affecting the precision of dimensions and the quality of surface finish. In real-world applications, flank wear serves as the primary indicator for evaluating tool wear.

<div style="text-align: center;">
    <img src="Images/image.png?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

The flank wear, as depicted in the image, stands out as the most commonly referenced wear parameter in monitoring machining processes. Both the average and maximum values of flank wear play crucial roles in assessing tool life. These metrics provide valuable insights into the tool's performance and help determine when maintenance or replacement is necessary.

## 1.2. Tool wear inspection
There are two primary methods for inspecting tool wear:

* Direct Measurement: This method involves physically measuring the wear on the tool using specialized equipment such as calipers, micrometers, or optical devices. Direct measurement provides precise data but may require pausing the machining process for inspection.

<div style="text-align: center;">
    <img src="Images/image-6.png?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* Indirect Measurement: In this method, tool wear is inferred based on other observable parameters such as cutting forces, vibrations, surface roughness, or changes in cutting sound. Indirect measurement techniques are often non-intrusive and can be performed while the machining process is ongoing, allowing for real-time monitoring of tool wear. However, they may be less accurate than direct measurements.

<div style="text-align: center;">
    <img src="Images/image-5.png?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

# 2. Data collection

In this project, we utilize a direct inspection method employing an optical sensor, specifically a smart camera integrated into the milling machine, for tool wear inspection. In the image below, you can observe the different parts of the camera, as well as the cutting tool.
<div style="text-align: center;">
    <img src="Images/image-8.png?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

The objective is to conduct milling operations using the cutting tool on a C45 workpiece under various parameters and observe the wear over the course of cutting time. The video depicts a single test session capturing the milling process.

<div style="text-align: center;">
    <img src="Images/image-9.gif?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

After each milling operation, the cutting tool will reposition itself and move in front of the camera to capture images of all teeth (4 images).

<div style="text-align: center;">
    <img src="Images/image-7.gif?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

After collecting the images, the next step is to detect wear on the cutting tool. To achieve this, we require a segmentation algorithm. However, before training the algorithm, we need to generate a training dataset. Since supervised learning method is preferred for this task, we must start by manually labeling the wear areas in the images. There are several open-source software options available for this purpose, but I recommend using the Image Labeler tool in MATLAB.


# 3. Image Labelling

## 3.1. Download training data set

To begin our exploration into image labelling, we'll start by downloading a set of training images. You can find the dataset containing 35 images at this link: https://github.com/EhsanWBK/KIPII

## 3.2. Image labelling

Now, let's dive into the process of labelling images using the Image Labeler app in MATLAB. This step-by-step guide will help you annotate the wear sites within the images effectively.

* Open MATLAB and Launch the Image Labeler App:
    * Navigate to the APPS tab.
    * Click on Image Labeler.
* Import Images:
    * In the Label Toolbar within the Image Labeler app, select Import.
    * Choose From File and select the images you downloaded earlier.
* Define Pixel Labels:
    * In the ROI Labels section, select Label.
    * Choose Pixel label and add a label name (e.g., "Wear Site").
* Annotation Process:
    * Use the Polygon tool to mark the wear areas within the images accurately.
* Save Your Progress:
    * Save the session by clicking on the save icon or using the shortcut Ctrl + S.
    * This will create an imageLabellingSession.mat file, preserving your annotations.
* Export Labelled Images:
    * Once you've finished annotating all the images, it's time to export the ground truth data.
    * In the Label Toolbar, select Export to File.
<div style="text-align: center;">
    <img src="Images/image-10.png?raw=1" alt="Alt text" style="display: block; margin: 0 auto;">
</div>


## 3.3. Converting Ground Truth to Mask
To convert the ground truth annotations into masks, we will apply linearly distributed thresholds to obtain the original class values. This process allows us to translate the labelled regions into binary masks, facilitating further analysis and model training.

* TASK:

In [12]:
#import libraries
import os
import cv2
import numpy as np
from pathlib import Path

# Ordnerpfade
read_directory = "../images/raw_labels"
write_directory = "../images/raw_labels_binary"
num_classes = 2

# Zielordner erstellen
os.makedirs(write_directory, exist_ok=True)

# Dataset size
img_name_list = sorted(os.listdir(read_directory))
img_name_list = [img for img in img_name_list if  "Label" in img and "image" in img]
print('Dataset size: ', len(img_name_list))

# Find unique colors and list them
uniqueColors = set()
for img_name in img_name_list:
    img_path = os.path.join(read_directory, img_name)
    img = cv2.imread(img_path, flags=0)  # Open image in greyscale mode
    uniqueColors |= set(np.unique(img))
print("Found Colors (unique greyscale values [0..255]): ", len(uniqueColors))

# Categorize using thresholds
minimum = int(min(uniqueColors))
maximum = int(max(uniqueColors))
print('Minimum Threshold: ' + str(minimum))
print('Maximum Threshold: ' + str(maximum))

# Generate linearly distributed thresholds
thresholds = np.linspace(start=minimum, stop=maximum + 1, num=num_classes + 1)

# Generate linearly distributed classes [0..255]
colorClasses = np.linspace(start=0, stop=255, num=num_classes)
print("New greyscale values: ", colorClasses)

# Apply thresholds on masks
for img_name in img_name_list:

    img_path = os.path.join(read_directory, img_name)
    img = cv2.imread(img_path, flags=0)

    # Leeres Ausgabebild erzeugen
    new_img = np.zeros_like(img, dtype=np.uint8)

    # Schwellenwert-Klassifizierung
    for i in range(num_classes):
        lower = thresholds[i]
        upper = thresholds[i + 1]
        mask = (img >= lower) & (img < upper)
        new_img[mask] = colorClasses[i]

    # Ergebnisbild speichern
    out_path = os.path.join(write_directory, img_name)
    cv2.imwrite(out_path, new_img)


Dataset size:  35
Found Colors (unique greyscale values [0..255]):  111
Minimum Threshold: 0
Maximum Threshold: 255
New greyscale values:  [  0. 255.]
